In [9]:
# 중심축 기반 부피 계산
import trimesh
import numpy as np
from scipy.spatial.transform import Rotation
from scipy.optimize import least_squares

# 상수 정의


THICKNESS = 0.0003 ##단면의 두께
MIN_POINTS_PER_SLICE = 10 ##원 검출 시 최소 포인트 수
MAX_CONSECUTIVE_FAILURES = 10 ##원 검출 시 최대 연속 실패 횟수
MIN_RADIUS, MAX_RADIUS = 0.02, 0.1 ##원 검출 시 최소 및 최대 반지름
MIN_POINTS_PER_METER = 50.0 ##중심축 기반 원검출시 최소 포인트 수
MIN_ANGLE_COVERAGE_THRESHOLD = 0.2  # 원 검출 시 최소 각도 커버리지 (20% 이상)



# 새로운 바닥 검출 파라미터
BOTTOM_SEARCH_RADIUS = 0.02   # 중심축으로부터 2cm 범위 (20mm)
BOTTOM_GRID_SIZE = 20          # 그리드 20x20 = 400개 셀
MIN_BOTTOM_GRID_COVERAGE = 0.2  # 바닥 검출을 위한 최소 그리드 커버리지 (0.0 ~ 1.0, 10% 이상)
BOTTOM_SEARCH_RATIO = 0.3     # 하위 30%만 바닥 검출

# 중심축 선정을 위한 지표 가중치
WEIGHT_PPM = 0
WEIGHT_ANGLE_COVERAGE = 0.5
WEIGHT_GRID_COVERAGE = 0.5
WEIGHT_INLIER_RATIO = 0

# 그리드 기반 커버리지 계산 파라미터
GRID_SIZE = 40  
INNER_RADIUS_RATIO_FOR_GRID = 0.8  # 원 반지름의 60% 이내

# 각도 커버리지 계산 파라미터
ANGLE_SEGMENTS = 180  

# GLB 로드 및 z축이 중력방향이 되도록 scene을 회전하는 함수
def load_and_align_scene(filepath):
    scene = trimesh.load(filepath)
    A = np.array(scene.metadata.get('hf_alignment', np.eye(4))).reshape(4, 4)
    gravity = (A[:3, :3] @ np.array([[1,0,0],[0,-1,0],[0,0,-1]]) @ np.array([0,1,0]))
    gravity = gravity / np.linalg.norm(gravity)
    
    R = Rotation.align_vectors([[0, 0, 1]], [gravity])[0]
    T = np.eye(4)
    T[:3, :3] = R.as_matrix()
    for geom in scene.geometry.values():
        geom.apply_transform(T)
    
    for geometry in scene.geometry.values():
        if isinstance(geometry, trimesh.PointCloud):
            return geometry.vertices
    return None


# 원 검출 함수들

def calculate_ppm(inlier_points, radius):
    """둘레당 인라이너 포인트 수 계산 (Points Per Meter)"""
    if radius <= 0:
        return 0.0
    circumference = 2 * np.pi * radius
    n_inliers = len(inlier_points)
    ppm = n_inliers / circumference if circumference > 0 else 0
    return ppm

def calculate_angle_coverage(inlier_points, center):
    """인라이너가 원 둘레를 커버하는 정도 계산 (0.0 ~ 1.0)"""
    if len(inlier_points) == 0:
        return 0.0
    
    # 각 포인트의 각도 계산
    relative_points = inlier_points - center
    angles = np.arctan2(relative_points[:, 1], relative_points[:, 0])
    
    # 각도를 0~2π 범위로 정규화
    angles = (angles + 2 * np.pi) % (2 * np.pi)
    
    # 각도 구간으로 나누기
    segment_size = 2 * np.pi / ANGLE_SEGMENTS
    segment_indices = (angles / segment_size).astype(int)
    segment_indices = np.clip(segment_indices, 0, ANGLE_SEGMENTS - 1)
    
    # 커버된 구간 수 계산
    covered_segments = len(np.unique(segment_indices))
    coverage = covered_segments / ANGLE_SEGMENTS
    
    return coverage

def calculate_grid_coverage(points_2d, center, radius, inner_radius_ratio=INNER_RADIUS_RATIO_FOR_GRID, grid_size=GRID_SIZE):
    """그리드 기반 면적 커버리지 계산 (0.0 ~ 1.0, 낮을수록 좋음)"""
    inner_radius = radius * inner_radius_ratio
    
    # 내부 포인트 필터링
    distances = np.linalg.norm(points_2d - center, axis=1)
    inner_points = points_2d[distances < inner_radius]
    
    if len(inner_points) == 0:
        return 0.0  # 완전히 비어있음 (최고)
    
    # 그리드 범위 계산
    x_min, x_max = center[0] - inner_radius, center[0] + inner_radius
    y_min, y_max = center[1] - inner_radius, center[1] + inner_radius
    
    # 그리드 셀 크기
    cell_size_x = (x_max - x_min) / grid_size
    cell_size_y = (y_max - y_min) / grid_size
    
    if cell_size_x <= 0 or cell_size_y <= 0:
        return 0.0
    
    # 각 포인트가 속한 그리드 셀 인덱스 계산
    grid_x = ((inner_points[:, 0] - x_min) / cell_size_x).astype(int)
    grid_y = ((inner_points[:, 1] - y_min) / cell_size_y).astype(int)
    
    # 그리드 범위 내로 클리핑
    grid_x = np.clip(grid_x, 0, grid_size - 1)
    grid_y = np.clip(grid_y, 0, grid_size - 1)
    
    # 포인트가 있는 셀 찾기
    occupied_cells = set(zip(grid_x, grid_y))
    coverage = len(occupied_cells) / (grid_size * grid_size)
    
    return coverage

# 인라이너 포인트들을 받아서 원의 중심과 반지름을 정확하게 추정하는 함수
def refine_circle_ls(inlier_points):
    if len(inlier_points) < 3:
        return None
    center_init = inlier_points.mean(axis=0)
    radius_init = np.mean(np.linalg.norm(inlier_points - center_init, axis=1))
    
    def residuals(params):
        cx, cy, r = params
        return np.linalg.norm(inlier_points - [cx, cy], axis=1) - r
    
    try:
        result = least_squares(residuals, [center_init[0], center_init[1], radius_init],
                              bounds=([-0.5, -0.5, 0.001], [0.5, 0.5, 0.2]))
        cx, cy, r = result.x
        if 0.01 < r < 0.15:
            return {'center': np.array([cx, cy]), 'radius': r}
    except:
        pass
    return None

# 랜덤 샘플링을 통해 원을 추정하는 함수.한 단면에서 최적의 원을 찾을 때는 단순히 가장 원다운 원(인라이너 비율)을 찾음.
# 여러 단면에서 나온 원들 중에에 중심축을 추정하기 위한 단면을 고를때는 원점과 원중심까지의 거리를 고려해서 상위 5개의 단면의 원들의 중앙값 사용용
def fit_circle_ransac(points_2d, n_iter=200, threshold=0.005, min_inliers=10):
    if len(points_2d) < 3:
        return None
    
    # PPM 제외한 가중치 정규화 (합이 1이 되도록)
    sum_without_ppm = WEIGHT_ANGLE_COVERAGE + WEIGHT_GRID_COVERAGE + WEIGHT_INLIER_RATIO
    weight_angle_norm = WEIGHT_ANGLE_COVERAGE / sum_without_ppm
    weight_grid_norm = WEIGHT_GRID_COVERAGE / sum_without_ppm
    weight_inlier_norm = WEIGHT_INLIER_RATIO / sum_without_ppm
    
    best_circle, best_score = None, -1.0
    for _ in range(n_iter):
        p1, p2, p3 = points_2d[np.random.choice(len(points_2d), 3, replace=False)]
        try:
            A = np.array([[2*(p2[0]-p1[0]), 2*(p2[1]-p1[1])],
                         [2*(p3[0]-p1[0]), 2*(p3[1]-p1[1])]])
            b = np.array([p2[0]**2 - p1[0]**2 + p2[1]**2 - p1[1]**2,
                         p3[0]**2 - p1[0]**2 + p3[1]**2 - p1[1]**2])
            center = np.linalg.solve(A, b)
            radius = np.linalg.norm(p1 - center)
            
            if not (0.01 < radius < 0.15):
                continue
            
            distances = np.abs(np.linalg.norm(points_2d - center, axis=1) - radius)
            inliers = distances < threshold
            n_inliers = np.sum(inliers)
            
            if n_inliers < min_inliers:
                continue
            
            # refine
            refined = refine_circle_ls(points_2d[inliers])
            if refined:
                # 인라이너 포인트 추출
                inlier_points = points_2d[inliers]
                inlier_ratio = n_inliers / len(points_2d)
                
                # 지표 계산 (정규화 불필요, 이미 0~1 범위)
                angle_coverage = calculate_angle_coverage(inlier_points, refined['center'])
                grid_coverage = calculate_grid_coverage(points_2d, refined['center'], refined['radius'])
                grid_coverage_inverted = 1.0 - grid_coverage  # 반전 (낮을수록 좋음)
                
                # 곱셈 방식으로 점수 계산 (가중치를 지수로 사용)
                # 0 방지를 위해 작은 값 추가
                score = ((inlier_ratio + 1e-8) ** weight_inlier_norm * 
                        (angle_coverage + 1e-8) ** weight_angle_norm * 
                        (grid_coverage_inverted + 1e-8) ** weight_grid_norm)
                
                if score > best_score:
                    best_score = score
                    # PPM은 나중에 계산 (중심축 선정용)
                    ppm = calculate_ppm(inlier_points, refined['radius'])
                    best_circle = {
                        **refined, 
                        'score': inlier_ratio,  # 기존 호환성
                        'n_inliers': n_inliers,
                        'ppm': ppm,
                        'angle_coverage': angle_coverage,
                        'grid_coverage': grid_coverage
                    }
        except np.linalg.LinAlgError:
            continue
    return best_circle

def detect_circle_at_axis(slice_2d, center_axis, n_iter=30, threshold=0.0025, prefer_inner=True, max_radius=None): # 원둘레 중위 2.5mm는 인라이어어
    """중심축 기반으로 원 검출 함수. ANGLE_COVERAGE로 원 존재 여부를 판별합니다."""
    if len(slice_2d) < MIN_POINTS_PER_SLICE:
        return None
    
    # 거리 계산
    distances = np.linalg.norm(slice_2d - center_axis, axis=1)
    
    # 최대 반지름 제한으로 포인트 미리 필터링 (계산량 감소)
    if max_radius is not None:
        filtered_mask = distances <= max_radius
        filtered_slice_2d = slice_2d[filtered_mask]
        filtered_distances = distances[filtered_mask]
        
        if len(filtered_slice_2d) < MIN_POINTS_PER_SLICE:
            return None
    else:
        filtered_slice_2d = slice_2d
        filtered_distances = distances
    
    best_radius, best_score = None, -1.0
    
    for _ in range(n_iter):
        # 필터링된 포인트에서만 후보 선택
        r_candidate = filtered_distances[np.random.randint(len(filtered_slice_2d))]
        if not (MIN_RADIUS <= r_candidate <= MAX_RADIUS):
            continue
        
        # 필터링된 포인트로 인라이너 계산
        inlier_count = np.sum(np.abs(filtered_distances - r_candidate) < threshold)
        circumference = 2 * np.pi * r_candidate
        ppm = inlier_count / circumference if circumference > 0 else 0
        
        # 내벽 선호: 작은 반지름에 가중치 부여
        if prefer_inner:
            # 반지름이 작을수록 높은 가중치 (MAX_RADIUS 기준으로 정규화)
            radius_weight = 1.0 + (MAX_RADIUS - r_candidate) / MAX_RADIUS
            score = ppm * radius_weight
        else:
            score = ppm
        
        if score > best_score:
            best_score, best_radius = score, r_candidate
    
    # 최종 검증: ANGLE_COVERAGE로 원 존재 여부 판별
    if best_radius:
        # 인라이너 포인트 추출 (필터링된 포인트 사용)
        inlier_mask = np.abs(filtered_distances - best_radius) < threshold
        inlier_points = filtered_slice_2d[inlier_mask]
        
        if len(inlier_points) >= MIN_POINTS_PER_SLICE:
            # ANGLE_COVERAGE 계산
            angle_coverage = calculate_angle_coverage(inlier_points, center_axis)
            
            # ANGLE_COVERAGE 임계값으로 판별
            if angle_coverage >= MIN_ANGLE_COVERAGE_THRESHOLD:
                return best_radius
    
    return None

def check_has_bottom(slice_2d, center_axis, radius, threshold=0.0003):
    """
    바닥이 있는지 확인하는 함수 (그리드 기반 커버리지 방식)
    
    Parameters:
    - slice_2d: 2D 단면 포인트 배열 (N, 2)
    - center_axis: 중심축 좌표 (x, y)
    - radius: 원의 반지름 (사용하지 않음, 호환성을 위해 유지)
    - threshold: 사용하지 않음 (호환성을 위해 유지)
    
    Returns:
    - bool: 바닥이 있으면 True, 없으면 False
    """
    if len(slice_2d) < MIN_POINTS_PER_SLICE:
        return False
    
    # 중심축으로부터 BOTTOM_SEARCH_RADIUS 범위 내 포인트 필터링
    distances = np.linalg.norm(slice_2d - center_axis, axis=1)
    bottom_mask = distances <= BOTTOM_SEARCH_RADIUS
    bottom_points = slice_2d[bottom_mask]
    
    if len(bottom_points) < MIN_POINTS_PER_SLICE:
        return False  # 포인트가 부족하면 바닥 없음
    
    # 그리드 범위 계산 (중심축 기준 ±BOTTOM_SEARCH_RADIUS)
    x_min = center_axis[0] - BOTTOM_SEARCH_RADIUS
    x_max = center_axis[0] + BOTTOM_SEARCH_RADIUS
    y_min = center_axis[1] - BOTTOM_SEARCH_RADIUS
    y_max = center_axis[1] + BOTTOM_SEARCH_RADIUS
    
    # 그리드 셀 크기 계산
    cell_size_x = (x_max - x_min) / BOTTOM_GRID_SIZE
    cell_size_y = (y_max - y_min) / BOTTOM_GRID_SIZE
    
    if cell_size_x <= 0 or cell_size_y <= 0:
        return False
    
    # 각 포인트가 속한 그리드 셀 인덱스 계산
    grid_x = ((bottom_points[:, 0] - x_min) / cell_size_x).astype(int)
    grid_y = ((bottom_points[:, 1] - y_min) / cell_size_y).astype(int)
    
    # 그리드 범위 내로 클리핑
    grid_x = np.clip(grid_x, 0, BOTTOM_GRID_SIZE - 1)
    grid_y = np.clip(grid_y, 0, BOTTOM_GRID_SIZE - 1)
    
    # 포인트가 있는 셀 찾기 (중복 제거)
    occupied_cells = set(zip(grid_x, grid_y))
    
    # 전체 그리드 셀 개수
    total_cells = BOTTOM_GRID_SIZE * BOTTOM_GRID_SIZE
    
    # 그리드 커버리지 계산 (포인트가 있는 셀 비율)
    grid_coverage = len(occupied_cells) / total_cells
    
    # 바닥 판정: 그리드 커버리지가 임계값 이상이면 바닥 있음
    has_bottom = grid_coverage >= MIN_BOTTOM_GRID_COVERAGE
    
    return has_bottom

# 유틸리티 함수들
def get_slice(points, z, thickness=THICKNESS):
    """높이 z에서 슬라이스 추출"""
    mask = (points[:, 2] >= z - thickness) & (points[:, 2] <= z + thickness)
    return points[mask]

def add_volume_data(volume_data, z, radius):
    """volume_data에 단면 정보 추가"""
    volume_data.append({'z': z, 'radius': radius, 'area': np.pi * radius**2})

def search_heights(points, z_range, center_axis, volume_data, max_failures=MAX_CONSECUTIVE_FAILURES, max_radius=None):
    """높이 범위를 탐색하며 원 검출"""
    failures = 0
    for z in z_range:
        slice_points = get_slice(points, z)
        if len(slice_points) < MIN_POINTS_PER_SLICE:
            failures += 1
            if failures >= max_failures:
                break
            continue
        
        radius = detect_circle_at_axis(slice_points[:, :2], center_axis, max_radius=max_radius)
        if radius is not None:
            failures = 0
            add_volume_data(volume_data, z, radius)
        else:
            failures += 1
            if failures >= max_failures:
                break



# 메인 로직
points = load_and_align_scene('/data/ephemeral/home/project/output/scene.glb')

# 전처리: 원점 근처 필터링 및 정렬
mask = np.linalg.norm(points[:, :2], axis=1) < 0.5 #원점기준 xy거리 0.2m 이하 필터링
filtered = points[mask]
sorted_idx = np.argsort(filtered[:, 2])
sorted_points, sorted_heights = filtered[sorted_idx], filtered[sorted_idx, 2]

# 1단계: 중심축 계산
z_range = np.linspace(sorted_heights.min(), sorted_heights.max(), 150) #z축 범위 150개 구간으로 나누기.
circle_data = []

for z in z_range:
    start = np.searchsorted(sorted_heights, z - THICKNESS)
    end = np.searchsorted(sorted_heights, z + THICKNESS)
    if end - start < MIN_POINTS_PER_SLICE:
        continue
    
    slice_2d = sorted_points[start:end, :2]
    # 원 검출 (새로운 지표 포함)
    circle = fit_circle_ransac(slice_2d, n_iter=50, threshold=0.005, min_inliers=10)
    
    if circle and 'ppm' in circle and 'angle_coverage' in circle and 'grid_coverage' in circle:
        circle_data.append({
            'z': z,
            'center': circle['center'],
            'radius': circle['radius'],
            'score': circle['score'],
            'ppm': circle['ppm'],
            'angle_coverage': circle['angle_coverage'],
            'grid_coverage': circle['grid_coverage']
        })

if not circle_data:
    print("컵 원을 찾지 못했습니다.")
else:
    # 정규화를 위한 최대/최소값 계산 (PPM만 정규화 필요)
    all_ppms = [c['ppm'] for c in circle_data]
    ppm_min, ppm_max = min(all_ppms), max(all_ppms)
    
    # 각 원에 대해 점수 계산
    for circle in circle_data:
        # PPM만 정규화 (큰 값이므로)
        ppm_norm = (circle['ppm'] - ppm_min) / (ppm_max - ppm_min + 1e-8)
        
        # 나머지 지표는 정규화 없이 바로 사용 (이미 0~1 범위)
        inlier_ratio = circle['score']  # 이미 0~1
        angle_coverage = circle['angle_coverage']  # 이미 0~1
        grid_coverage_inverted = 1.0 - circle['grid_coverage']  # 반전 (낮을수록 좋음)
        
        # 곱셈 방식으로 점수 계산 (가중치를 지수로 사용)
        # 0 방지를 위해 작은 값 추가
        final_score = ((ppm_norm + 1e-8) ** WEIGHT_PPM * 
                      (inlier_ratio + 1e-8) ** WEIGHT_INLIER_RATIO * 
                      (angle_coverage + 1e-8) ** WEIGHT_ANGLE_COVERAGE * 
                      (grid_coverage_inverted + 1e-8) ** WEIGHT_GRID_COVERAGE)
        
        circle['final_score'] = final_score
    
    # final_score 기준으로 정렬하여 가장 높은 점수의 단면 선택
    circle_data.sort(key=lambda x: x['final_score'], reverse=True)
    best_circle = circle_data[0]  # 점수가 가장 높은 단면
    center_axis = best_circle['center']  # 가장 높은 점수 단면의 중심을 중심축으로 사용
    reference_z = best_circle['z']  # 부피 계산 할때 처음 탐색을 시작할 z값. final score가장 높은 단면
    reference_radius = best_circle['radius']  # 중심축 원의 반지름 저장
    max_radius_limit = reference_radius * 1.15  # 115% 제한값 계산
    
    print(f"중심축: ({center_axis[0]:.6f}, {center_axis[1]:.6f}), 기준 높이: {reference_z:.4f}m, 기준 반지름: {reference_radius:.4f}m")
    print(f"최대 반지름 제한: {max_radius_limit:.4f}m (기준 반지름의 115%)")
    
    # 2단계: 부피 계산을 위한 원 검출
    volume_data = []
    z_min, z_max = points[:, 2].min(), points[:, 2].max()
    STEP = 0.002  # 2mm 간격
    
    # 기준 높이에서 원 검출 시도
    ref_slice = get_slice(points, reference_z)
    if len(ref_slice) >= MIN_POINTS_PER_SLICE:
        ref_radius = detect_circle_at_axis(ref_slice[:, :2], center_axis, max_radius=max_radius_limit)
        if ref_radius:
            add_volume_data(volume_data, reference_z, ref_radius)
    
    # 위쪽 탐색: 기준 높이 + 0.002, + 0.004, ... (z_max까지)
    z_above = np.arange(reference_z + STEP, z_max + STEP, STEP)
    search_heights(points, z_above, center_axis, volume_data, max_radius=max_radius_limit)
    
    # 아래쪽 탐색: 기준 높이 - 0.002, - 0.004, ... (z_min까지)
    z_below = np.arange(reference_z - STEP, z_min - STEP, -STEP)
    search_heights(points, z_below, center_axis, volume_data, max_radius=max_radius_limit)
    


    # 3단계: 하위 20% 범위에서만 바닥 검출
    if volume_data:
        volume_data.sort(key=lambda x: x['z'])
        
        # 높이 범위 계산
        height_range = volume_data[-1]['z'] - volume_data[0]['z']
        bottom_20_percent_range = height_range * BOTTOM_SEARCH_RATIO
        bottom_20_end_z = volume_data[0]['z'] + bottom_20_percent_range
        
        print(f"\n바닥 검출 범위: {volume_data[0]['z']:.4f}m ~ {bottom_20_end_z:.4f}m (하위 {BOTTOM_SEARCH_RATIO*100:.0f}%)")
        
        # 하위 20% 범위의 데이터만 바닥 검출
        bottom_detected_heights = []
        for data in volume_data:
            z = data['z']
            # 하위 20% 범위에 있는지 확인
            if z <= bottom_20_end_z:
                radius = data['radius']
                slice_points = get_slice(points, z)
                
                if len(slice_points) >= MIN_POINTS_PER_SLICE:
                    has_bottom = check_has_bottom(slice_points[:, :2], center_axis, radius)
                    if has_bottom:
                        bottom_detected_heights.append(z)
        
        # 바닥이 있는 가장 높은 높이 찾기
        if bottom_detected_heights:
            bottom_end_z = max(bottom_detected_heights)
            print(f"바닥 검출: {len(bottom_detected_heights)}개 높이에서 바닥 발견")
            print(f"바닥 끝 높이: {bottom_end_z:.4f}m")
            
            # 바닥이 아닌 부분만 필터링 (바닥 끝 높이보다 위쪽만)
            volume_data_filtered = [data for data in volume_data if data['z'] > bottom_end_z]
            
            print(f"필터링 전: {len(volume_data)}개 단면")
            print(f"필터링 후: {len(volume_data_filtered)}개 단면 (바닥 제외)")
            
            if len(volume_data_filtered) > 1:
                volume_data = volume_data_filtered
            else:
                print("경고: 바닥을 제외하면 단면이 부족합니다. 전체 데이터 사용.")
        else:
            print("하위 20% 범위에서 바닥이 검출되지 않았습니다. 전체 데이터로 부피 계산합니다.")


    # 4단계: 부피 계산
    if volume_data:
        volume_data.sort(key=lambda x: x['z'])
        total_volume = sum((volume_data[i]['area'] + volume_data[i+1]['area']) / 2 * 
                          (volume_data[i+1]['z'] - volume_data[i]['z']) 
                          for i in range(len(volume_data) - 1))
        
        print(f"\n단면 개수: {len(volume_data)}개")
        print(f"높이: {volume_data[0]['z']:.4f}m ~ {volume_data[-1]['z']:.4f}m")
        print(f"부피: {total_volume * 1000:.2f} mL ({total_volume * 1e6:.2f} cm³)")
    else:
        print("원을 찾지 못했습니다.")

중심축: (-0.016401, 0.152596), 기준 높이: 0.1207m, 기준 반지름: 0.0554m
최대 반지름 제한: 0.0637m (기준 반지름의 115%)

바닥 검출 범위: -0.0533m ~ 0.0013m (하위 30%)
바닥 검출: 1개 높이에서 바닥 발견
바닥 끝 높이: -0.0493m
필터링 전: 92개 단면
필터링 후: 89개 단면 (바닥 제외)

단면 개수: 89개
높이: -0.0473m ~ 0.1287m
부피: 1.29 mL (1293.79 cm³)


In [5]:
#ARcore 좌표계로 변환하는 함수

import json

def load_jsonl(jsonl_path):
    """JSONL 파일 읽기 - 여러 줄에 걸친 JSON 객체 처리"""
    data = []
    with open(jsonl_path, 'r') as f:
        content = f.read()
    
    # 중괄호 카운팅으로 각 JSON 객체 분리
    current_obj = ""
    brace_count = 0
    for char in content:
        current_obj += char
        if char == '{':
            brace_count += 1
        elif char == '}':
            brace_count -= 1
            if brace_count == 0:  # JSON 객체 완성
                try:
                    data.append(json.loads(current_obj.strip()))
                except json.JSONDecodeError as e:
                    print(f"JSON 파싱 오류: {e}")
                current_obj = ""
    return data

# ============================================
# 중력 정렬 씬 좌표계 → ARCore 월드 좌표계 변환 함수
# ============================================
def transform_point_to_arcore(point_scene, first_frame_params, scene_metadata=None):
    """
    중력 정렬 씬 좌표계의 점을 ARCore 월드 좌표계로 변환
    
    변환 경로:
    1. 중력 정렬 씬 → hf_alignment 적용 씬 (T_inv)
    2. hf_alignment 적용 씬 → 첫 프레임 카메라 기준 좌표계 (T_camera_origin)
    3. 첫 프레임 카메라 기준 좌표계 → ARCore 월드 좌표계 (c2w)
    
    Args:
        point_scene: 중력 정렬 씬 좌표계의 점 [x, y, z]
        first_frame_params: 첫 프레임의 ARCore pose 정보 (quat, pos 포함)
        scene_metadata: 씬 메타데이터 (hf_alignment 포함)
    
    Returns:
        ARCore 월드 좌표계의 점 [x, y, z] (OpenGL 좌표계)
    """
    import numpy as np
    from scipy.spatial.transform import Rotation
    
    point_scene = np.array(point_scene, dtype=np.float32)
    
    if scene_metadata is None:
        raise ValueError("scene_metadata is required")
    
    A = np.array(scene_metadata.get('hf_alignment', np.eye(4))).reshape(4, 4)
    
    # 중력 벡터 계산 (hf_alignment 적용 씬 좌표계에서)
    gl_to_cv = np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]])
    arcore_y = np.array([0, 1, 0])
    gravity = (A[:3, :3] @ gl_to_cv @ arcore_y)
    gravity = gravity / np.linalg.norm(gravity)
    
    # 중력 정렬 회전 행렬 계산
    R = Rotation.align_vectors([[0, 0, 1]], [gravity])[0]
    T = np.eye(4)
    T[:3, :3] = R.as_matrix()
    T_inv = np.linalg.inv(T)
    
    # 1단계: 중력 정렬 역변환 (중력 정렬 씬 → hf_alignment 적용 씬)
    point_after_inv = T_inv[:3, :3] @ point_scene + T_inv[:3, 3]
    
    # 첫 프레임 카메라의 c2w 계산
    quat = np.array(first_frame_params['quat'])
    pos_arcore = np.array(first_frame_params['pos'])
    
    gl_to_cv = np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]], dtype=np.float32)
    R_c2w_gl = Rotation.from_quat(quat).as_matrix()
    R_c2w_cv = gl_to_cv @ R_c2w_gl @ gl_to_cv.T
    pos_cv = gl_to_cv @ pos_arcore
    
    c2w = np.eye(4, dtype=np.float32)
    c2w[:3, :3] = R_c2w_cv
    c2w[:3, 3] = pos_cv
    
    w2c0 = np.linalg.inv(c2w)
    
    # M 행렬 (CV → glTF 좌표계 변환)
    M = np.eye(4, dtype=np.float64)
    M[1, 1] = -1.0
    M[2, 2] = -1.0
    M_inv = np.linalg.inv(M)
    
    # A_no_center = M @ w2c0
    A_no_center = M @ w2c0.astype(np.float64)
    A_no_center_inv = np.linalg.inv(A_no_center)
    
    # c2w0 계산 (원본 씬 좌표계에서의 첫 프레임 카메라 → 월드)
    c2w0 = A_no_center_inv @ M
    
    # 첫 프레임 카메라 위치 (hf_alignment 적용 씬 좌표계에서)
    first_camera_pos_original = c2w0[:3, 3]  # 원본 씬 좌표계
    first_camera_pos_hf_aligned = A[:3, :3] @ first_camera_pos_original + A[:3, 3]
    
    # 2단계: 첫 프레임 카메라 기준 좌표계로 변환 (glTF 좌표계)
    # 첫 프레임 카메라 위치를 원점으로 이동
    point_camera_frame_gltf = point_after_inv - first_camera_pos_hf_aligned
    
    # glTF → OpenCV 좌표계 변환 (c2w가 OpenCV 기준이므로)
    point_camera_frame_cv = M_inv[:3, :3] @ point_camera_frame_gltf
    
    # 3단계: ARCore 월드 좌표계로 변환 (OpenCV 좌표계 기준)
    point_arcore_world_cv = c2w[:3, :3] @ point_camera_frame_cv + c2w[:3, 3]
    
    # OpenCV → OpenGL 좌표계 변환 (ARCore 월드 좌표계)
    cv_to_gl = np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]], dtype=np.float32)
    point_arcore_world_gl = cv_to_gl @ point_arcore_world_cv
    
    return point_arcore_world_gl

In [ ]:
#메인로직
import trimesh
import numpy as np
import glob
import os
point_scene = np.array([-0.016401, 0.152596, 0.1287])  # [x, y, z]


# 1. GLB 파일 로드 (씬 메타데이터 포함)
scene = trimesh.load('/data/ephemeral/home/project/output/scene.glb')
scene_metadata = scene.metadata  # hf_alignment 포함

# 2. JSONL 파일 로드 (ARCore pose 데이터)
# ar_folder 내 가장 최근 session 폴더 찾기
base_folder = "/data/ephemeral/home/project/ar_folder"
session_folders = sorted(glob.glob(os.path.join(base_folder, "session_*")))

if not session_folders:
    raise FileNotFoundError(f"세션 폴더를 찾을 수 없습니다: {os.path.join(base_folder, 'session_*')}")

session_folder = session_folders[-1]  # 가장 최근 세션 사용
print(f"사용할 세션 폴더: {session_folder}")

# JSONL 파일 찾기 (세션 폴더 안에서)
jsonl_files = glob.glob(os.path.join(session_folder, "*.jsonl"))
if not jsonl_files:
    raise FileNotFoundError(f"JSONL 파일을 찾을 수 없습니다: {os.path.join(session_folder, '*.jsonl')}")

if len(jsonl_files) > 1:
    print(f"경고: 여러 개의 JSONL 파일이 발견되었습니다. 첫 번째 파일을 사용합니다: {jsonl_files[0]}")

jsonl_path = jsonl_files[0]
print(f"사용할 JSONL 파일: {jsonl_path}")

jsonl_data = load_jsonl(jsonl_path)

# 3. 첫 번째 프레임 pose 가져오기 (시간순 정렬)
jsonl_data_sorted = sorted(jsonl_data, key=lambda x: x['t_ns'])
first_frame_params = jsonl_data_sorted[0]

# 4. 씬 좌표계의 점 (중력 정렬된 씬 좌표계)


# 5. ARCore 월드 좌표계로 변환
point_arcore_world = transform_point_to_arcore(
    point_scene=point_scene,
    first_frame_params=first_frame_params,
    scene_metadata=scene_metadata
)

print(f"씬 좌표계: {point_scene}")
print(f"ARCore 월드 좌표계: {point_arcore_world}")

사용할 세션 폴더: /data/ephemeral/home/ar_folder/session_coor1
사용할 JSONL 파일: /data/ephemeral/home/ar_folder/session_coor1/session_1768192999669.jsonl
씬 좌표계: [-0.016401  0.152596  0.1287  ]
ARCore 월드 좌표계: [-0.00545869 -0.37039563 -0.26400851]
